# Preprocess

# Setup

## Install library

In [12]:
# !pip install xlrd
# !pip install openpyxl
# !pip install yfinance

## Import library

In [13]:
# System
import random
import os
import shutil
import time
import datetime as dt

# Network
import requests

# Data Structure
import pandas as pd

# Data source
import yfinance as yf

# User defined
import constants

## Define parameter

Retrieve from *constants.py*

In [14]:
# Trading days
start_date = constants.start_date
end_date = constants.end_date

# Directory
data_dir_url = constants.data_dir_url

# CSV file list
csv_file_list = [
                    constants.inst_buy_file, constants.inst_sell_file, 
                    constants.retail_buy_file, constants.retail_sell_file
                ]

Get every Monday with the given range

In [15]:
trading_date_list = pd.date_range(start=start_date, end=end_date, freq='W-MON')
trading_date_list

DatetimeIndex(['2019-01-07', '2019-01-14', '2019-01-21', '2019-01-28',
               '2019-02-04', '2019-02-11', '2019-02-18', '2019-02-25',
               '2019-03-04', '2019-03-11',
               ...
               '2022-08-29', '2022-09-05', '2022-09-12', '2022-09-19',
               '2022-09-26', '2022-10-03', '2022-10-10', '2022-10-17',
               '2022-10-24', '2022-10-31'],
              dtype='datetime64[ns]', length=200, freq='W-MON')

# Fetch data

In [16]:
def parse_excel(excel_res, inst_header=4, retail_header=17):
    inst_data = pd.read_excel(excel_res, header=inst_header, nrows=10)
    retail_data = pd.read_excel(excel_res, header=retail_header, nrows=10)

    inst_buy = inst_data.iloc[:, :3]
    inst_sell = inst_data.iloc[:, 3:6]

    retail_buy = retail_data.iloc[:, :3]
    retail_sell = retail_data.iloc[:, 3:6]

    return inst_buy, inst_sell, retail_buy, retail_sell

In [17]:
def write_csv(df: pd.DataFrame, file_path: str, published_date: dt):
    # standardise the column name
    df.columns = ['stock', 'stock code', 'net amount (S$M)']

    # standised to yfinance stock code format
    df['stock code'] = df['stock code'].apply(lambda ticker: str(ticker) + '.SI')

    # add date
    df['published date'] = published_date.strftime('%Y-%m-%d')

    # if file not exist then add the header
    is_file_exist = not os.path.exists(file_path)
    df.to_csv(file_path, index=False, mode='a', header=is_file_exist)

In [18]:
file_has_postfix = {
    '2019-02-11': '_3',
    '2019-04-01': '_0',
    '2019-04-15': '_0',
    '2019-06-10': '_0',
    '2019-06-24': '_0',
    '2019-09-16': '_2',
    '2019-10-28': '_0',
    '2019-11-25': '_0',
    '2019-12-09': '_0',
    '2019-12-16': '_0',
    '2020-02-24': '_2',
    '2020-04-27': '_0',
    '2020-05-11': '_0',
    '2020-11-23': '_0',
    '2020-11-30': '_0',
    '2021-12-27': '_0',
    '2021-01-10': '_0',
    '2021-01-31': '_0',
    '2022-03-07': '_0',
}

def gen_url(directory_date: dt, file_date: dt):
    year_month = directory_date.strftime('%Y-%m')  # Eg. 2022-03
    day_month_year = str(file_date.day)
    postfix = file_has_postfix.get(file_date.strftime('%Y-%m-%d'), '')
    base_url = ''

    # day_month_year
    if (file_date.year <= 2019) or \
        (file_date.year == 2022 and (file_date.month == 6 or file_date.month == 7)):
        day_month_year += file_date.strftime(' %B %Y')  # Eg. 3 June 2022
    else:
        day_month_year += file_date.strftime(' %b %Y')  # Eg. 3 Oct 2022
    
    # base url
    if file_date.year <= 2019:
        base_url = 'SGX Institutional and Retail fund flow weekly tracker '
    else:
        base_url = 'SGX Fund Flow Weekly Tracker '

    url = f'https://api2.sgx.com/sites/default/files/{year_month}/{base_url}(Week of {day_month_year}){postfix}.xlsx'
    
    return url

In [19]:
def slow_api_req():
    time.sleep(3 * random.random())

In [20]:
# fresh start the data directory
if os.path.exists(data_dir_url):
    shutil.rmtree(data_dir_url)

os.mkdir(data_dir_url)

# fetch data and preprocess the data
for date in trading_date_list:
    directory_date = date
    file_date = date - dt.timedelta(days=7)  # Refer to last week data

    n_days = 1

    while True:
        try:
            url = gen_url(directory_date, file_date)
            print(url)
            

            req = requests.get(url)
            assert req.status_code != 404

            excel_file = req.content

            inst_retail_buy_sell_df = parse_excel(excel_file)
            
            for df, filepath in zip(inst_retail_buy_sell_df, csv_file_list):
                write_csv(df, filepath, date)

            break
        except AssertionError:
            # if Monday is holiday, increase by T + 1 day
            print('### ERROR: ' + file_date.strftime('%Y-%m-%d'))
            file_date = file_date + dt.timedelta(days=1)
        
        n_days += 1

        # after 5 business day, terminate the call
        if n_days >= 5:
            break

        slow_api_req()

https://api2.sgx.com/sites/default/files/2019-01/SGX Institutional and Retail fund flow weekly tracker (Week of 31 December 2018).xlsx
https://api2.sgx.com/sites/default/files/2019-01/SGX Institutional and Retail fund flow weekly tracker (Week of 7 January 2019).xlsx
https://api2.sgx.com/sites/default/files/2019-01/SGX Institutional and Retail fund flow weekly tracker (Week of 14 January 2019).xlsx
https://api2.sgx.com/sites/default/files/2019-01/SGX Institutional and Retail fund flow weekly tracker (Week of 21 January 2019).xlsx
https://api2.sgx.com/sites/default/files/2019-02/SGX Institutional and Retail fund flow weekly tracker (Week of 28 January 2019).xlsx
https://api2.sgx.com/sites/default/files/2019-02/SGX Institutional and Retail fund flow weekly tracker (Week of 4 February 2019).xlsx
https://api2.sgx.com/sites/default/files/2019-02/SGX Institutional and Retail fund flow weekly tracker (Week of 11 February 2019)_3.xlsx
https://api2.sgx.com/sites/default/files/2019-02/SGX Instit

# Data Cleaning

## Update ticker

For company that has update their ticker, we need to update their old ticker to the latest ticker

In [21]:
constants.updated_ticker

{'41A.SI': '8K7.SI',
 'O32.SI': 'VC2.SI',
 'A68U.SI': 'HMN.SI',
 'CNNU.SI': 'CWBU.SI',
 'CH8.SI': 'QES.SI'}

In [22]:
updated_ticker = constants.updated_ticker

for url in csv_file_list:
    df = pd.read_csv(url)
    df['stock code'] = df['stock code'].apply(lambda ticker: updated_ticker[ticker] if ticker in updated_ticker.keys() else ticker)

    df.to_csv(url, index=False)

## Remove delisted stock

In [23]:
def get_stock_ticker(url):
    df = pd.read_csv(url)
    tickers = df['stock code'].unique().tolist()

    return tickers

In [24]:
def remove_delisted_stock(url, delieted_ticker):
    df = pd.read_csv(url)
    delisted_stock_bool = df['stock code'].isin(delieted_ticker)

    delisted_stock_bool = df['stock code'].isin(delieted_ticker)
    df = df[~delisted_stock_bool]

    return df

In [25]:
# Get all ticker exist in data
tickers = []

for url in csv_file_list:
    tickers.extend(get_stock_ticker(url))

tickers = set(tickers)

# Find delisted stock
delisted_tickers = []

for ticker in tickers:
    hist = yf.Ticker(ticker).history()

    if len(hist) == 0:
        delisted_tickers.append(ticker)

# Filter and update the csv
for url in csv_file_list:
    df = remove_delisted_stock(url, delisted_tickers)
    df.to_csv(url, index=False)

T39.SI: No data found, symbol may be delisted
ADQU.SI: No data found, symbol may be delisted
F25U.SI: No data found, symbol may be delisted
B16.SI: No data found, symbol may be delisted
J37.SI: No data found, symbol may be delisted
5ME.SI: No data found, symbol may be delisted
RF1U.SI: No data found, symbol may be delisted
PGU.SI: No data found, symbol may be delisted
588.SI: No data found, symbol may be delisted
5H0.SI: No data found, symbol may be delisted
C61U.SI: No data found, symbol may be delisted
VL6.SI: No data found, symbol may be delisted
K2LU.SI: No data found, symbol may be delisted
C31.SI: No data found, symbol may be delisted
CTN.SI: No data found, symbol may be delisted
U04.SI: No data found, symbol may be delisted
BQO.SI: No data found, symbol may be delisted
AYL.SI: No data found, symbol may be delisted
CEE.SI: No data found, symbol may be delisted
K11.SI: No data found, symbol may be delisted
ND8U.SI: No data found, symbol may be delisted
B2F.SI: No data found, symbo

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a59e5772-0ffb-4811-b384-2b5a9f52032a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>